In [6]:
import os
import sys
%pylab

import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    %pylab
from src import data

from src import efficient_frontier
#from src import filter_assets
from src import plot
from src import portfolio
from src import capm
import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds
import datetime
from dateutil import rrule

Using matplotlib backend: TkAgg
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


c:\Users\Tor Osted\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['rrule', 'plot']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [7]:
x = pd.Timestamp(np.datetime64('2000-01-01'))
x1 = pd.Timestamp(np.datetime64('2022-01-01'))

In [8]:
weights = np.array([1/3,1/3,1/3])
min_esg_score = 1400
max_esg_score = 2000
df = pd.read_excel(r"C:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\data\ESG_DATA_S&P500.xlsx")
dates = [x,x1]

start_year = dates[0]
end_year = dates[1]

Bounds1 = Bounds(-1,2) #How willing we are to go short and to invest in one particular stock
Wanted_return = 0.20 #Only used when using the wanted_return constraint
maximum_risk = 0.10 #Either used when using maximum risk constraint or cmle portfolio
sharpe_type = "No_extra_constraint"  # rename to constraint, can be either Wanted_return or Maximum_risk or No_extra_constraint
risk_free_rate = 0.01
covariance_window = 5 #has to be in years



esg_data = data.esg_score_weight(df,weights,min_esg_score,max_esg_score) #ESG_DATA filtering function
full_data = data.stock_monthly_close(esg_data,dates) #Gives us the data we need from the stocks with relevant ESG_scores
prices,esgdata = data.seperate_full_data(full_data) #Gives us price data and esg data to use in future functions
pct_returns = data.pct_returns_from_prices(prices) #Gives us the returns in pct for our stocks
parameters = portfolio.efficient_frontier_solo(pct_returns,Bounds1, sharpe_type,start_year,end_year, Wanted_return, maximum_risk,'yearly') #Different amount of rows from ESG_DATA as it drops all na. Gives us the efficient frontier for the given tie frame #Different amount of rows from ESG_DATA as it drops all na. Gives us the efficient frontier for the given tie frame
weights_of_each_portfolio = efficient_frontier.weights_of_portfolio(prices,parameters) #Gives us the weights of the individual stocks in our portfolio
portfolio_esg_score = portfolio.esg_score_of_portfolio(weights_of_each_portfolio,esgdata.head(1)) #gives us the ESG score of our portfolio
portfolio_allocations = portfolio.capital_mark_line_returns(parameters,risk_free_rate,maximum_risk)[1] #How much we allocate to our market portfolio compared to the risk-free asset
cmle_returns = portfolio.capital_mark_line_returns(parameters,risk_free_rate,maximum_risk)[0] #Return based on how much we allocate to the risk free asset.
sp500 = data.data_for_beta(prices)
pct_returns_sp500 = data.pct_returns_from_prices(sp500,)
betas_of_portfolio = capm.calculate_portfolio_beta(pct_returns_sp500,prices,weights_of_each_portfolio,'SPY')
capm_expected_return = capm.capm_calc(pct_returns_sp500['SPY'].mean(),risk_free_rate,betas_of_portfolio)

strategy1 = [df,
             weights,
             min_esg_score,
             max_esg_score,
             Bounds1,
             sharpe_type,
             x,
             x1,
             Wanted_return,
             maximum_risk,
             'monthly',
             risk_free_rate]






Min. Risk = 27.534% => Return: 15.192%  Sharpe Ratio = 0.55
Max. Sharpe Ratio = 0.66 => Return: 21.71%  Risk: 32.914%
Excpected return on investment is 121.43209898270383%


In [22]:
# import packages
from datetime import date
from dateutil.relativedelta import relativedelta


def backtesting(strategy,monthly_or_yearly_rebalancing,rebalancing_freq,start_date,end_date,covariance_window,market_name):
    #start = 2015
   # end = 2022

    covariance_window_time_delta = relativedelta(years=covariance_window) #The time delta for the covariance window
    #print([start_date-covariance_window_time_delta,end_date])
    esg_data = data.esg_score_weight(strategy[0],strategy[1],strategy[2],strategy[3])
    full_data = data.stock_monthly_close(esg_data,[pd.Timestamp(start_date-covariance_window_time_delta),pd.Timestamp(end_date)])
    prices,esgdata = data.seperate_full_data(full_data)
    pct_returns = data.pct_returns_from_prices(prices)

    
    listparameters = []
    list_of_port_weights =[]
    list_of_port_esg_scores = []
    list_of_port_allocations = []
    list_of_cmle_returns = []
    list_of_portfolio_actual_returns = []
    sp500_list = []
    list_of_pct_returns_sp500 = []
    betas_of_portfolios = []
    capm_for_portfolio = []
    i = 0
    if monthly_or_yearly_rebalancing == 'yearly':
        delta = relativedelta(years=rebalancing_freq)
        
        while (start_date <= end_date):
            #print(start_date-covariance_window_time_delta,start_date)
            
            

            listparameters.append(portfolio.efficient_frontier_solo(pct_returns,
                            Bounds1,
                            sharpe_type,
                            start_date-covariance_window_time_delta,#The start date for portfolio optimization will be the start date minus the covariance window
                            start_date,#The end date for portfolio optimization will be the start date minus the covariance window
                            Wanted_return,
                            maximum_risk,
                            strategy[10])) 
            list_of_port_weights.append(efficient_frontier.weights_of_portfolio(prices,listparameters[i]))
            list_of_port_esg_scores.append(portfolio.esg_score_of_portfolio(list_of_port_weights[i],esgdata.head(1)))
            list_of_port_allocations.append(portfolio.capital_mark_line_returns(listparameters[i],risk_free_rate,maximum_risk)[1])
            list_of_cmle_returns.append(portfolio.capital_mark_line_returns(listparameters[i],risk_free_rate,maximum_risk)[0])
           # print(list_of_port_weights)
            #list_of_portfolio_actual_returns.append(list_of_port_weights[i]*prices[])

            sp500_list.append(data.data_for_beta(prices,[market_name],(start_date-covariance_window_time_delta,start_date)))
            print(start_date-covariance_window_time_delta,start_date)
            print(sp500_list)
           # betas_of_portfolios.append(capm.calculate_portfolio_beta(sp500_list[i],pct_returns,list_of_port_weights[i],market_name))
           # capm_for_portfolio.append(capm.capm_calc(sp500_list[i][market_name].mean(),strategy1[11],list_of_port_weights[i]))


            start_date += delta
            i += 1

        return(list_of_port_weights,list_of_port_esg_scores,list_of_port_allocations,list_of_cmle_returns,betas_of_portfolios)

    elif monthly_or_yearly_rebalancing == 'monthly':
        delta = delta = np.timedelta64(rebalancing_freq,'M')
        while (start_date <= end_date):
            
            
            

            listparameters = listparameters.append(portfolio.efficient_frontier_solo(pct_returns,
                            Bounds1,
                            sharpe_type,
                            start_date-covariance_window_time_delta,#The start date for portfolio optimization will be the start date minus the covariance window
                            start_date,#The end date for portfolio optimization will be the start date minus the covariance window
                            Wanted_return,
                            maximum_risk,
                            strategy[10]))
            print(listparameters[i]) 
            list_of_port_weights.append(efficient_frontier.weights_of_portfolio(prices,listparameters[i]))
            list_of_port_esg_scores.append(portfolio.esg_score_of_portfolio(list_of_port_weights[i],esgdata.head(1)))
            list_of_port_allocations.append(portfolio.capital_mark_line_returns(listparameters[i],risk_free_rate,maximum_risk)[1])
            list_of_cmle_returns.append(portfolio.capital_mark_line_returns(listparameters[i],risk_free_rate,maximum_risk)[0])
            #sp500_list.append(data.data_for_beta(prices,[market_name],(start_date-covariance_window_time_delta,start_date)))
            #print(start_date-covariance_window_time_delta,start_date)
            #print(sp500_list)
            #betas_of_portfolios.append(capm.calculate_portfolio_beta(data.pct_returns_from_prices(sp500_list[i]),prices,weights_of_each_portfolio,market_name))
            #capm_for_portfolio.append(capm.capm_calc(data.pct_returns_from_prices(sp500_list[i][market_name]).mean(),strategy[11],betas_of_portfolios[i]))


        return(list_of_port_weights,list_of_port_esg_scores,list_of_port_allocations,list_of_cmle_returns)

    
    else:
        print('You can only call this function with monthly or yearly')
        return(False)
    

backtesting(strategy1,'yearly',1,datetime.datetime(2015,1,1),datetime.datetime(2022,1,1),10,'SPY')
#backtesting(strategy1,'yearly',1,np.datetime64('2015-01-01'),np.datetime64('2022-01-01'),10) #US dates YYYY-MM-DD


#This makes each portfolio using strategy 1 rebalancing every 6 months from 2015 until 2022, using a covariance window of ten years

#This makes each portfolio for 


Min. Risk = 6.540% => Return: 0.888%  Sharpe Ratio = 0.14
Max. Sharpe Ratio = 0.21 => Return: 2.10%  Risk: 10.057%


c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\data.py:116: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  stock_data = pd.concat([stock_data[dates[0]:dates[1]],stock_data_download], axis = 1)


2005-01-01 00:00:00 2015-01-01 00:00:00
[                                AMZN       MSFT         SPY
Date                                                       
2005-01-01 00:00:00-05:00   2.161000  26.280001  118.160004
2005-02-01 00:00:00-05:00   1.759000  25.160000  120.629997
2005-03-01 00:00:00-05:00   1.713500  24.170000  117.959999
2005-04-01 00:00:00-05:00   1.618000  25.299999  115.750000
2005-05-01 00:00:00-04:00   1.775500  25.799999  119.480003
...                              ...        ...         ...
2014-09-01 00:00:00-04:00  16.122000  46.360001  197.020004
2014-10-01 00:00:00-04:00  15.273000  46.950001  201.660004
2014-11-01 00:00:00-04:00  16.931999  47.810001  207.199997
2014-12-01 00:00:00-05:00  15.517500  46.450001  205.539993
2015-01-01 00:00:00-05:00  17.726500  40.400002         NaN

[121 rows x 3 columns]]
Excpected return on investment is          AMZN        MSFT
0  217.783941  152.658391%
Min. Risk = 6.876% => Return: 1.230%  Sharpe Ratio = 0.18
Max. Shar

c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\data.py:116: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  stock_data = pd.concat([stock_data[dates[0]:dates[1]],stock_data_download], axis = 1)


2006-01-01 00:00:00 2016-01-01 00:00:00
[                                AMZN       MSFT         SPY
Date                                                       
2005-01-01 00:00:00-05:00   2.161000  26.280001  118.160004
2005-02-01 00:00:00-05:00   1.759000  25.160000  120.629997
2005-03-01 00:00:00-05:00   1.713500  24.170000  117.959999
2005-04-01 00:00:00-05:00   1.618000  25.299999  115.750000
2005-05-01 00:00:00-04:00   1.775500  25.799999  119.480003
...                              ...        ...         ...
2014-09-01 00:00:00-04:00  16.122000  46.360001  197.020004
2014-10-01 00:00:00-04:00  15.273000  46.950001  201.660004
2014-11-01 00:00:00-04:00  16.931999  47.810001  207.199997
2014-12-01 00:00:00-05:00  15.517500  46.450001  205.539993
2015-01-01 00:00:00-05:00  17.726500  40.400002         NaN

[121 rows x 3 columns],                                 AMZN       MSFT         SPY
Date                                                       
2006-01-01 00:00:00-05:00   2.2410

...

2010-01-01 00:00:00 2020-01-01 00:00:00
[                                AMZN       MSFT         SPY
Date                                                       
2005-01-01 00:00:00-05:00   2.161000  26.280001  118.160004
2005-02-01 00:00:00-05:00   1.759000  25.160000  120.629997
2005-03-01 00:00:00-05:00   1.713500  24.170000  117.959999
2005-04-01 00:00:00-05:00   1.618000  25.299999  115.750000
2005-05-01 00:00:00-04:00   1.775500  25.799999  119.480003
...                              ...        ...         ...
2014-09-01 00:00:00-04:00  16.122000  46.360001  197.020004
2014-10-01 00:00:00-04:00  15.273000  46.950001  201.660004
2014-11-01 00:00:00-04:00  16.931999  47.810001  207.199997
2014-12-01 00:00:00-05:00  15.517500  46.450001  205.539993
2015-01-01 00:00:00-05:00  17.726500  40.400002         NaN

[121 rows x 3 columns],                                 AMZN       MSFT         SPY
Date                                                       
2006-01-01 00:00:00-05:00   2.2410

c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\data.py:116: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  stock_data = pd.concat([stock_data[dates[0]:dates[1]],stock_data_download], axis = 1)


2011-01-01 00:00:00 2021-01-01 00:00:00
[                                AMZN       MSFT         SPY
Date                                                       
2005-01-01 00:00:00-05:00   2.161000  26.280001  118.160004
2005-02-01 00:00:00-05:00   1.759000  25.160000  120.629997
2005-03-01 00:00:00-05:00   1.713500  24.170000  117.959999
2005-04-01 00:00:00-05:00   1.618000  25.299999  115.750000
2005-05-01 00:00:00-04:00   1.775500  25.799999  119.480003
...                              ...        ...         ...
2014-09-01 00:00:00-04:00  16.122000  46.360001  197.020004
2014-10-01 00:00:00-04:00  15.273000  46.950001  201.660004
2014-11-01 00:00:00-04:00  16.931999  47.810001  207.199997
2014-12-01 00:00:00-05:00  15.517500  46.450001  205.539993
2015-01-01 00:00:00-05:00  17.726500  40.400002         NaN

[121 rows x 3 columns],                                 AMZN       MSFT         SPY
Date                                                       
2006-01-01 00:00:00-05:00   2.2410

c:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\BachelorThesis\src\data.py:116: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  stock_data = pd.concat([stock_data[dates[0]:dates[1]],stock_data_download], axis = 1)


([       AMZN      MSFT
  0  0.828917  0.171083,
         AMZN      MSFT
  0  0.788442  0.211558,
         AMZN      MSFT
  0  0.901154  0.098846,
        AMZN     MSFT
  0  0.87337  0.12663,
         AMZN      MSFT
  0  0.681351  0.318649,
         AMZN      MSFT
  0  0.546494  0.453506,
         AMZN      MSFT
  0  0.427734  0.572266,
         AMZN      MSFT
  0  0.347438  0.652562],
 [   ESG_score_of_portfolio
  0             1474.363515,
     ESG_score_of_portfolio
  0              1485.33222,
     ESG_score_of_portfolio
  0             1454.787161,
     ESG_score_of_portfolio
  0             1462.316616,
     ESG_score_of_portfolio
  0              1514.35387,
     ESG_score_of_portfolio
  0             1550.900175,
     ESG_score_of_portfolio
  0             1583.084181,
     ESG_score_of_portfolio
  0             1604.844434],
 [[0.9943318331926159],
  [1.060512401221944],
  [1.02760180987172],
  [1.1516797188518604],
  [1.4436553351925445],
  [1.6224857685050174],
  [1.72560906

In [15]:
#5.807% ,5.712%

#Min. Risk = 5.654% => Return: 2.439%  Sharpe Ratio = 0.43
#Max. Sharpe Ratio = 0.44 => Return: 2.49%  Risk: 5.715%

portfolio.efficient_frontier_solo(pct_returns,Bounds1, sharpe_type, '2012-01-01 00:00:00','2022-01-01 00:00:00', Wanted_return,maximum_risk,'monthly')

Min. Risk = 5.654% => Return: 2.439%  Sharpe Ratio = 0.43
Max. Sharpe Ratio = 0.44 => Return: 2.49%  Risk: 5.715%


[(0.05715299854808578,
  0.024930824873805338,
  0.056535569912816386,
  0.024394955329190386,
  array([0.05660303, 0.0566054 , 0.05660793, 0.05661066, 0.05661359,
         0.05661675, 0.05662017, 0.05662389, 0.05662792, 0.05663232,
         0.05663713, 0.05664241, 0.05664822, 0.05665464, 0.05666177,
         0.05666971, 0.05667859, 0.05668859, 0.0566999 , 0.05671276,
         0.0567275 , 0.05674449, 0.05676424, 0.05678741, 0.05681483,
         0.05684765, 0.05688741, 0.05693623, 0.05699712, 0.05707443,
         0.05717463, 0.05730763, 0.0574892 , 0.05774537, 0.05812114,
         0.10848659, 0.10848659, 0.10848659, 0.10848659, 0.10848659,
         0.10848659, 0.10848659, 0.0749193 , 0.09031204, 0.14379213,
         0.14379213, 0.14379213, 0.14379213, 0.14379213, 0.14379213]),
  array([-0.3       , -0.29235117, -0.28470235, -0.27705352, -0.2694047 ,
         -0.26175587, -0.25410704, -0.24645822, -0.23880939, -0.23116057,
         -0.22351174, -0.21586291, -0.20821409, -0.20056526, -0.1